# Selecting Subsets of Data in Pandas

This notebook is also available as a [blog post on Medium](https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-39e811c81a0c).

## Part 2: Boolean Indexing
This is part 2 of a seven-part series on how to select subsets of data from a pandas DataFrame or Series. Pandas offers a wide variety of options for subset selection which necessitates multiple articles. This series is broken down into the following 7 topics.

1. Selection with `[]`, `.loc` and `.iloc`
1. Boolean indexing
1. Assigning subsets of data
1. Selection with a MultiIndex
1. Selecting subsets of data with methods
1. Selections with other Index types
1. Internals, Miscellaneous, and Conclusion

## Part 1 vs Part 2 subset selection

Part 1 of this series covered subset selection with `[]`, `.loc` and `.iloc`. All three of these **indexers** use either the row/column labels or their integer location to make selections. The actual **data** of the Series/DataFrame is not used at all during the selection. 

In Part 2 of this series, on **boolean indexing**, we will select subsets of data based on the actual values of the data in the Series/DataFrame and NOT on their row/column labels or integer locations. 


## Documentation on boolean selection
I will always recommend reading the official documentation in addition to this tutorial when learning about boolean selection. The documentation uses more formal examples with dummy data, but is still an excellent resource.

The documentation use the term **boolean indexing** but you will also see **boolean selection**.

[Boolean Indexing from pandas documentation](http://pandas.pydata.org/pandas-docs/stable/indexing.html#boolean-indexing)

## Stack Overflow Data

The data that we will use for this tutorial comes from [Stack Overflow's data explorer](https://data.stackexchange.com/stackoverflow/query/new), which is a fantastic tool to gather an incredible amount of data from the site. You must know SQL in order to use the data explorer. The data explorer allows you to save queries. [Take a look at the query](http://data.stackexchange.com/stackoverflow/query/768430/get-all-questions-and-answerers-from-tag) I used to collect the data. 

The table below contains data on each question asked on stack overflow [tagged as pandas](https://stackoverflow.com/questions/tagged/pandas). 

The first question was asked March 30, 2011. Since then, more than 56,000 questions have been added as of December 2, 2017.

In [2]:
import pandas as pd
import numpy as np

In [3]:
so = pd.read_csv('../../data/stackoverflow_qa.csv')
so.head()

FileNotFoundError: File b'../../data/stackoverflow_qa.csv' does not exist

## Asking simple questions in plain English

Before we get to the technical definition of boolean indexing, let's see some examples of the types of questions it can answer.

* Find all questions that were created before 2014
* Find all questions with a score more than 50
* Find all questions with a score between 50 and 100
* Find all questions answered by Scott Boston
* Find all questions answered by the following 5 users
* Find all questions that were created between March, 2014 and October 2014 that were answered by Unutbu and have score less than 5.
* Find all questions that have score between 5 and 10 or have a view count of greater than 10,000
* Find all questions that are not answered by Scott Boston

You will also see examples like this referred to by the term **queries**.

## All queries have criteria
Each of the above queries have a strict logical criteria that must be checked one row at a time.

## Keep or Discard entire row of data
If you were to manually answer the above queries, you would need to scan each row and determine whether the row as a whole meets the criterion or not. If the row meets the criteria, then it is kept and if not, then it is discarded.

## Each row will have a `True` or `False` value associated with it
When you perform boolean indexing, each row of the DataFrame (or value of a Series) will have a `True` or `False` value associated with it depending on whether or not it meets the criterion. True/False values are known as **boolean**. The documentation refers to the entire procedure as **boolean indexing**. 

Since we are using the booleans to select data, it is sometimes referred to as **boolean selection**. Essentially, we are using booleans to select subsets of data.

## Using `[]` and `.loc` for boolean selection
We will use the same three indexers, **`[]`** and **`.loc`** from part 1 to complete our boolean selections. We will do so by placing a sequence of booleans inside of these indexer. The sequence will be the same number of rows/values as the DataFrame/Series it is doing the selection on.

The **`.iloc`** indexer can be made to work with boolean selection but is almost never used. A small section towards the end will show why it's unnecessary.

## Focus on `[]` for now
To simplify things, we will only the brackets, **`[]`**, which I called **just the indexing operator** from part 1. We will get to the other indexers a bit later.

## Use a small DataFrame to get started
Before we make our first boolean selection, let's simplify matters and use the first five rows of the stack overflow data as our starting DataFrame.

In [ ]:
so_head = so.head()
so_head

## Manually create a list of booleans
For our first boolean selection, we will not answer any interesting 'English' queries and instead just select rows with a list of booleans.

For instance, let's select the first and third rows by creating the following list:

In [ ]:
criteria = [True, False, True, False, False]

We can pass this list of booleans to just the indexing operator and complete our selection:

In [ ]:
so_head[criteria]

## Wait a second... Isn't `[]` just for column selection?
The primary purpose of *just the indexing operator* for a DataFrame is to select one or more columns by using either a string or a list of strings. Now, all of a sudden, this example is showing that entire rows are selected with boolean values. This is what makes pandas, unfortunately, one of the most confusing libraries to use. 

## Operator Overloading
*Just the indexing* operator is overloaded. This means, that depending on the inputs, pandas will do something completely different. Here are the rules for the different objects you pass to *just the indexing operator*.
* string - return a column as a Series
* list of strings - return all those columns as a DataFrame
* a slice - select rows (can do both label and integer location - confusing!)
* a sequence of booleans - select all rows where **`True`**

In summary, primarily *just the indexing operator* selects **columns**, but if you pass it a sequence of booleans it will select all **rows** that are **`True`**.

## What do you mean by 'sequence'?
I keep using the term **sequence of booleans** to refer to the `True/False` values. Technically, the most common built-in [Python sequence](https://docs.python.org/3/library/stdtypes.html#typesseq) types are lists and tuples. In addition to a list, you will most often be using a pandas Series as your 'sequence' of booleans.

Let's manually create a boolean Series to select the last three rows of **`so_head`**.

In [ ]:
s = pd.Series([False, False, True, True, True])
s

In [ ]:
so_head[s]

## Take care when creating a boolean Series by hand
The above example only worked because the index of both the boolean Series and **`so_head`** were the exact same. Let's output them so you can clearly see this.

In [ ]:
s.index

In [ ]:
so_head.index

## Boolean selection fails when the index doesn't align
When you are using a boolean Series to do boolean selection, the index of both objects must be the exact same. Let's create a slightly different Series with a different index than the DataFrame it is indexing on.

In [ ]:
s = pd.Series([False, False, True, True, True], index=[2, 3, 4, 5, 6])
s

In [ ]:
so_head[s]

## `IndexingError`: Unalignable boolean Series!
If the index of both the boolean Series and the object you are doing boolean selection on don't match exactly, you will get the above error. This is one reason, as you will below, why you will almost never create boolean Series by hand like this.

## Also use NumPy arrays
You can also use NumPy arrays to do boolean selection. NumPy arrays have no index so you won't get the error above, but your array needs to be the same exact length as the object you are doing boolean selection on.

In [ ]:
a = np.array([True, False, False, True, False])
so_head[a]

## Never creating boolean Series by hand
You will likely never create a boolean Series by hand as was done above. Instead, you will produce them based on the values of your data.

## Use the comparison operators to create boolean Series
The primary method of creating a Series of booleans is to use one of the six comparison operators: 
* **`<`**
* **`<=`**
* **`>`**
* **`>=`**
* **`==`**
* **`!=`** 

## Use comparison operator with a single column of data
You will almost always use the comparison operators on just a single column or Series of data. For instance, let's create a boolean Series from the **`score`** column. Let's determine if the score is at least 10.

We select the score column and then test the condition that each value is greater than or equal to 10. Notice that this operations gets applied to each value in the Series. A boolean Series is returned.

In [4]:
criteria = so['score'] >= 10
criteria.head(10)

NameError: name 'so' is not defined

## Finally making a boolean selection
Now that we have our boolean Series stored in the variable **`criteria`**, we can pass this to *just the indexing operator* to select only the rows that have a score of at least 10. 

We are going to use the entire **`so`** DataFrame for the rest of the tutorial.

In [ ]:
so_score_10_or_more = so[criteria]
so_score_10_or_more.head()

## How many rows have a score of at least ten
Just by looking at the head of the resulting DataFrame, we don't know how many rows passed our criterion. Let's output the shape of both our original and our resulting DataFrame.

In [ ]:
so.shape

In [ ]:
so_score_10_or_more.shape

Only about 3% of questions get a score of 10 or more.

## Boolean selection in one line
Often, you will see boolean selection happen in a single line of code instead of the multiple lines we used above. If the following is confusing for you, then I recommend storing your boolean Series to a variable like I did with **`criteria`** above.

It is possible to put the creation of the boolean Series inside of *just the indexing operator* like this.

In [ ]:
so[so['score'] >= 10].head()

## Single condition expression
Our first example tested a single condition (whether the score was 10 or more). Let's test a different single condition and look for all the questions that are answered by **Scott Boston**. The **`ans_name`** variable holds the display names of the people who posted the accepted answer to the question.

We use the **`==`** operator to test for equality and again store this result to the variable **`criteria`**. Again, we pass this variable to *just the indexing operator* which completes our selection.

In [ ]:
# step 1 - create boolean Series
criteria = so['ans_name'] == 'Scott Boston'

# step 2 - do boolean selection
so[criteria].head()

## Multiple condition expression
So far, both our boolean selections have involved a single condition. You can, of course, have as many conditions as you would like. To do so, you will need to combine your boolean expressions using the three logical operators **and**, **or** and **not**.

## Use `&`, `|`, `~`
Although Python uses the syntax **`and`**, **`or`**, and **`not`**, these will not work when testing multiple conditions with pandas. The details of why this is so will be left for part 7 of the Series.

You must use the following operators with pandas:
* **`&`** for **and**
* **`|`** for **or**
* **`~`** for **not**

## Our first multiple condition expression
Let's find all the questions that have a score of at least 5 and are answered by Scott Boston. To begin, we will create two separate variable to hold each criteria.

In [ ]:
criteria_1 = so['score'] >= 5
criteria_2 = so['ans_name'] == 'Scott Boston'

We will then use the **and** operator, the ampersand **`&`**, to combine them

In [ ]:
criteria_all = criteria_1 & criteria_2

We can now pass this final criteria to *just the indexing operator*

In [ ]:
so[criteria_all]

## Multiple conditions in one line
It is possible to combine the entire expression into a single line. Many pandas users like doing this, others hate it. Regardless, it is a good idea to know how to do so as you will definitely encounter it.

## Use parentheses to separate conditions
You must encapsulate each condition in a set of parentheses in order to make this work. This again, will be explained in part 7.

Each condition will be separated like this:
```Python
(so['score'] >= 5) & (so['ans_name'] == 'Scott Boston')
```

We can then drop this expression inside of *just the indexing operator*

In [ ]:
so[(so['score'] >= 5) & (so['ans_name'] == 'Scott Boston')]

## Using an `or` condition
Let's find all the questions that have a score of at least 100 or have at least 10 answers.

For the **or** condition, we use the pipe **`|`**

In [5]:
so[(so['score'] >= 100) | (so['answercount'] >= 10)].head()

NameError: name 'so' is not defined

## Reversing a condition with the `not` operator
The tilde character **`~`** represents the **not** operator and reverses a condition. For instance, if we wanted all the questions with score greater than 100, we could do it like this:

In [ ]:
so[~(so['score'] <= 100)].head()

Notice that there were parentheses around the condition '**`score`** less than equal to 100'. We had to use parentheses here or the operation wouldn't work correctly. 

Of course, this trivial example has no need for the not operator and can be replaced with the greater than operator, but it's easy to verify.

Let's look back up one example and invert the condition of **`score`** at least 100 or number of answers at least 10.  To do this, we will have to wrap our entire expression with parentheses like this:

```Python
~((so['score'] >= 100) | (so['answercount'] >= 10))
```

There is a set of parentheses around each inner expression as well.

## Complex conditions
It is possible to build extremely complex conditions to select rows of your DataFrame that meet a very specific criteria. For instance, we can select all questions answered by Scott Boston with **`score`** 5 or more OR questions answered by Ted Petrou with answer count 5 or more.

With multiple conditions, its probably best to break out the logic into multiple steps:

In [ ]:
criteria_1 = (so['score'] >= 5) & (so['ans_name'] == 'Scott Boston')
criteria_2 = (so['answercount'] >= 5) & (so['ans_name'] == 'Ted Petrou')
criteria_all = criteria_1 | criteria_2
so[criteria_all]

## Lots of `or` conditions in a single column - use `isin`
Occasionally, we will want to test equality in a single column to multiple values. This is most common in string columns. For instance, let's say we wanted to find all the questions answered by Scott Boston, Ted Petrou, MaxU, and unutbu.

One way to do this would be with four `or` conditions.

```Python
criteria = ((so['ans_name'] == 'Scott Boston') | (so['ans_name'] == 'Ted Petrou') | 
            (so['ans_name'] == 'MaxU') | (so['ans_name'] == 'unutbu'))
```

An easier way is to use the Series method **`isin`**. Pass it a list of all the items you want to check for equality.

In [ ]:
criteria = so['ans_name'].isin(['Scott Boston', 'Ted Petrou', 'MaxU', 'unutbu'])
criteria.head()

In [ ]:
so[criteria].head()

## Combining `isin` with other criteria
You can use the resulting boolean Series from the **`isin`** method in the same way you would from the logical operators. For instance, If we wanted to find all the questions answered by the people above and had score greater than 30 we would do the following:

In [ ]:
criteria_1 = so['ans_name'].isin(['Scott Boston', 'Ted Petrou', 'MaxU', 'unutbu'])
criteria_2 = so['score'] > 30
criteria_all = criteria_1 & criteria_2
so[criteria_all].tail()

## Use `isnull` to find rows with missing values
The **`isnull`** method returns a boolean Series where True indicates where a missing value is. For instance, questions that do not have an **accepted answer** have missing values for **`ans_name`**. Let's call **`isnull`** on this column.

In [ ]:
no_answer = so['ans_name'].isnull()
no_answer.head(6)

This is just another boolean Series which we can pass to *just the indexing operator*

In [ ]:
so[no_answer].head()

An alias of **`isnull`** is the **`isna`** method. Alias means it is the same exact method with a different name.

# Boolean Selection on a Series
All the examples thus far have taken place on the **`so`** DataFrame. Boolean selection on a Series happens almost identically. Since there is only one dimension of data, the queries you ask are usually going to be simpler.

First, let's select a single column of data as a Series such as the **`commentcount`** column.

In [ ]:
s = so['commentcount']
s.head()

Let's test for number of comments greater than 10

In [ ]:
criteria = s > 10
criteria.head()

Notice that there is no column selection here as we are already down to a single column. Let's pass this criteria to *just the indexing operator* to select just the values greater than 10.

In [ ]:
s[criteria].head()

We could have done this in one step like this

In [ ]:
s[s > 10].head()

If we wanted to find those comments greater than 10 but less than 15 we could have used an **and** condition like this:

In [ ]:
s[(s > 10) & (s < 15)].head()

## Another possibility is the `between` method
Pandas has lots of duplicate functionality built in to it. Instead of writing two boolean conditions to select all values inside of a range as was done above, you can use the **`between`** method to create a boolean Series. To use, pass it the left and right end points of the range. These endpoints are inclusive.

So, to replicate the previous example, you could have done this:

In [ ]:
s[s.between(11, 14)].head()

# Simultaneous boolean selection with rows and column labels with `.loc`
The **`.loc`** indexer was thoroughly covered in part 1 and will now be covered here to simultaneously select rows and columns. In part 1, it was stated that **`.loc`** made selections only by **label**. This wasn't strictly true as it is also able to do boolean selection along with selection by label.

Remember that **`.loc`** takes both a row selection and a column selection separated by a comma. Since the row selection comes first, you can pass it the same exact inputs that you do for *just the indexing operator* and get the same results.

Let's take a look at a couple examples from above:

In [ ]:
# same as above
so.loc[(so['score'] >= 5) & (so['ans_name'] == 'Scott Boston')]

In [ ]:
# same as above
criteria = so['ans_name'].isin(['Scott Boston', 'Ted Petrou', 'MaxU', 'unutbu'])
so.loc[criteria].head()

## Separate row and column selection with a comma for `.loc`
The great benefit of **`.loc`** is that it allows you to simultaneously do boolean selection along the rows and make column selections by label.

For instance, let's say we wanted to find all the questions with more than 20k views but only return the **`creationdate`**, **`viewcount`**, and **`ans_name`** columns. You would do the following.

In [ ]:
so.loc[so['viewcount'] > 20000, ['creationdate', 'viewcount', 'ans_name']].head(10)

You could have broken each selection into pieces like this:

```Python

row_selection = so['viewcount'] > 20000
col_selection = ['creationdate', 'viewcount', 'ans_name']
so.loc[row_selection, col_selection]
```

## Lots of combinations possible with `.loc`
Remember that **`.loc`** can take a string, a list of strings or a slice. You can use all three possible ways to select your data. You can also make very complex boolean selections for your rows.

Let's select rows with **`favoritecount`** between 30 and 40 and every third column beginning from **`title`** to the end.

In [ ]:
# weird but possible
so.loc[so['favoritecount'].between(30, 40), 'title'::3].head()

## Boolean selection for the columns?
It is actually possible to use a sequence of booleans to select columns. You pass a list, Series, or array of booleans the same length as the number of columns to **`.loc`**.

Let's do a simple manual example where we create a list of booleans by hand. First, let's find out how many columns are in our dataset

In [ ]:
so.shape

Let's create a list of 12 booleans

In [ ]:
col_bools = [True, False, False] * 4
col_bools

Use **`.loc`** to select all rows with just the `True` columns from **`col_bools`**.

In [ ]:
so.loc[:, col_bools].head()

You can simultaneously select rows and columns too. Let's select the same columns but for rows that have over 500,000 views.

In [ ]:
so.loc[so['viewcount'] > 500000, col_bools]

## A more practical example
Let's see a slightly more practical example of doing boolean selection on the columns. Let's say we flipped 10 coins one-hundred times and store each trial in a column in the DataFrame below

In [ ]:
coins = pd.DataFrame(np.random.randint(0, 2, (100, 10)), 
                     columns=list('abcdefghij'))
coins.head()

In [ ]:
coins.shape

If we are interested in selecting only the columns that have more than 50% heads, we could first take the mean of each column like this.

In [ ]:
coin_mean = coins.mean()
coin_mean

Let's test the condition that the percentage is greater than .5

In [ ]:
coin_mean > .5

Finally, we can use this boolean Series to select only the columns that meet our criteria.

In [ ]:
coins.loc[:, coins.mean() > .5].head()

# Column to column comparisons
All of the previous Series comparisons happened against a single scalar value. It is possible to create a boolean Series by comparing one column to another. For instance, we can find all the questions where there are more answers than **`score`**.

In [ ]:
criteria = so['answercount'] > so['score']
so[criteria].head()

In one line, the above would have looked like this:

```Python
so[so['answercount'] > so['score']]
```

## Almost never use `.iloc` with boolean selection
First, remember that **`.iloc`** uses INTEGER location to make its selections. 

You will rarely use **`.loc`** to do boolean selection and almost always use *just the indexing operator* or **`.loc`**. To see why, let's try and run a simple boolean selection to find all the rows that have more than 100,000 views.

In [ ]:
so.iloc[so['viewcount'] > 100000]

## `NotImplementedError`
The pandas developers have not decided to boolean selection (with a Series) for **`.iloc`** so it does not work. You can however convert the Series to a list or a NumPy array as a workaround.

Let's save our Series to a variable and double-check its type.

In [ ]:
criteria = so['viewcount'] > 100000
type(criteria)

Let's grab the underlying NumPy array with the **`values`** attribute and pass it to **`.iloc`**

In [ ]:
a = criteria.values
so.iloc[a].head()

You can make simultaneous column selection as well with integers.

In [ ]:
so.iloc[a, [5, 10, 11]].head()

I don't think I have ever used **`.iloc`** for boolean selection as its not implemented for Series. I added because it's one of the three main indexers in pandas and it's important to know that it's not used much at all for boolean selection.

## `.loc` and `[]` work the same on a Series for boolean selection
Boolean selection will work identically for **`.loc`** as it does with *just the indexing operator* on a Series. Both the indexers do row selection when passed a boolean Series. Since Series don't have columns, the two indexers are identical in this situation.

In [ ]:
s = so['score']

In [ ]:
s[s > 100].head()

In [ ]:
s.loc[s > 100].head()

# Summary

* **Boolean Indexing** or **Boolean Selection** is the selection of a subset of a Series/DataFrame based on the values themselves and not the row/column labels or integer location
* Boolean selection is used to answer common queries like "find all the female engineers with a salary over 150k/year"
* To do boolean selection, you first create a sequence of True/False values and pass it to a DataFrame/Series indexer
* Each row of data is kept or discarded
* The indexing operators are **overloaded** - change functionality depending on what is passed to them
* Typically, you will first create a boolean Series with one of the 6 comparison operators
* You will pass this boolean series to one of the indexers to make your selection
* Use the **`isin`** method to test for multiple equalities in the same column
* Use **`isnull`** to find all rows with missing values in a particular column
* Can use the **`between`** Series method to test whether Series values are within a range
* You can create complex criteria with the **and** (**`&`**), **or** (**`|`**), and **not** (**`~`**) logical operators
* When you have multiple conditions in a single line, you must wrap each expression with a parentheses
* If you have complex criteria, think about storing each set of criteria into its own variable (i.e. don't do everything in one line)
* If you are only selecting rows, then you will almost always use *just the indexing operator*
* If you are simultaneously doing boolean selection on the rows and selecting column labels then you will use **`.loc`**
* You will almost never use **`.iloc`** to do boolean selection
* Boolean selection works the same for Series as it does for DataFrames

# More to the story
Believe or not, there is still more to the story when it comes to boolean selection. We will cover more advanced topics in part 7.

# Exercises
Boolean selection is difficult at first and the syntax is somewhat clunky. It will take some time to master. These questions will start easy and progressively become more difficult.

## Data for exercises
We will use two datasets for these exercises. The stack overflow dataset which you have seen above and the employee dataset.

In [6]:
so = pd.read_csv('stackoverflow_qa.csv')
so.head()

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
0,5486226,2011-03-30 12:26:50,4,2113,Rolling median in python,3,4,1.0,yueerhu,125.0,Mike Pennington,26995.0
1,5515021,2011-04-01 14:50:44,8,7015,Compute a compounded return series in Python,3,6,7.0,Jason Strimpel,3301.0,Mike Pennington,26995.0
2,5558607,2011-04-05 21:13:50,2,7392,Sort a pandas DataMatrix in ascending order,2,0,1.0,Jason Strimpel,3301.0,Wes McKinney,43310.0
3,6467832,2011-06-24 12:31:45,9,13056,How to get the correlation between two timeser...,1,0,7.0,user814005,117.0,Wes McKinney,43310.0
4,7577546,2011-09-28 01:58:38,9,2488,"Using pandas, how do I subsample a large DataF...",1,0,5.0,Uri Laserson,958.0,HYRY,54137.0


In [7]:
employee = pd.read_csv('employee.csv')
employee.head()

,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,HIRE_DATE,JOB_DATE
0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,2006-06-12,2012-10-13
1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,2000-07-19,2010-09-18
2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,2015-02-03,2015-02-03
3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,1982-02-08,1991-05-25
4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,1989-06-19,1994-10-22


### Exercise 1
<span  style="color:green; font-size:16px">Find all the questions that have exactly 5 answers</span>

In [8]:
so[so['answercount'] == 5]

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
75,10373660,2012-04-29 16:10:35,199,207980,Converting a Pandas GroupBy object to DataFrame,5,0,90.0,saveenr,2421.0,Wes McKinney,43310.0
115,10791661,2012-05-29 00:06:52,7,12210,How do I discretize values in a pandas DataFra...,5,0,3.0,Uri Laserson,958.0,lbolla,4552.0
130,10972410,2012-06-10 21:12:43,19,46428,pandas: combine two columns in a DataFrame,5,0,5.0,BFTM,895.0,BrenBarn,136870.0
189,11391969,2012-07-09 09:04:33,44,27467,How to group pandas DataFrame entries by date ...,5,1,15.0,Boris Gorelik,9605.0,Wes McKinney,43310.0
246,11811392,2012-08-04 19:25:34,17,20051,How to generate a list from a pandas DataFrame...,5,0,6.0,turtle,1260.0,BrenBarn,136870.0
247,11824341,2012-08-06 07:47:34,0,1558,Pandas: date_range error,5,2,NaN,scry,781.0,scry,781.0
316,12190874,2012-08-30 06:12:46,59,61706,Pandas: Sampling a DataFrame,5,1,24.0,Blender,180189.0,Wouter Overmeire,20078.0
318,12200693,2012-08-30 15:45:26,36,31779,Python Pandas How to assign groupby operation ...,5,3,20.0,ely,26102.0,Wouter Overmeire,20078.0
335,12278347,2012-09-05 09:29:34,13,10908,How can I efficiently save a python pandas dat...,5,3,7.0,Griffith Rees,811.0,NaN,NaN
339,12307099,2012-09-06 19:32:25,89,56876,Modifying a subset of rows in a pandas dataframe,5,1,41.0,Arthur B.,977.0,BrenBarn,136870.0


### Exercise 2
<span  style="color:green; font-size:16px">Find all the questions that have less than 10 views</span>

In [9]:
so[so['viewcount'] < 10] 

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
7787,26493306,2014-10-21 18:06:10,1,6,How to convert hierarchical DataFrame back fro...,0,2,1.0,exp1orer,3551.0,NaN,NaN
17653,33641540,2015-11-10 23:19:36,0,9,Joining Dataframes in Pandas deletes an existi...,1,0,NaN,Rahul Biswas,88.0,NaN,NaN
29414,40062672,2016-10-15 18:24:21,-1,9,Replace one or more sub-strings from multiple ...,1,0,NaN,Andreuccio,186.0,NaN,NaN
36086,42538091,2017-03-01 17:20:37,0,9,Saving box plot pandas python,0,0,NaN,user2906657,119.0,NaN,NaN
36340,42080039,2017-02-07 01:13:30,0,8,pandas: count the non-duplicated elements when...,1,0,NaN,Edamame,2281.0,root,12202.0
40490,43271693,2017-04-07 07:04:19,0,9,Pandas Unsuccessfully Modify,1,1,NaN,Mellon,4.0,NaN,NaN
43489,45069021,2017-07-12 23:22:49,1,9,Can ask for strides of k subsequent items from...,1,0,NaN,user48956,3832.0,NaN,NaN
47371,45212468,2017-07-20 10:43:55,0,9,Pandas reads partial content,0,2,NaN,Keshav Reddy,16.0,NaN,NaN
49232,45890470,2017-08-25 23:16:57,0,7,result of pandas.PeriodIndex(astype(str)) in p...,1,0,NaN,checker,18.0,NaN,NaN
50461,45991431,2017-08-31 23:26:20,0,9,Check for excel file on a url link via while l...,0,0,NaN,Shyama Sonti,73.0,NaN,NaN


### Exercise 3
<span  style="color:green; font-size:16px">Find all the questions where the person asking it is the same as the person answering it</span>

In [10]:
so[so['quest_name'] == so['ans_name']]

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
10,8273092,2011-11-25 18:39:02,1,2333,python: pandas install errors,2,0,NaN,codingknob,2279.0,codingknob,2279.0
33,9721429,2012-03-15 14:08:31,9,5732,How do I read a fix width format text file in ...,4,2,NaN,user1234440,3369.0,user1234440,3369.0
58,10020591,2012-04-04 23:17:23,21,20171,How to resample a dataframe with different fun...,4,0,15.0,bmu,17129.0,bmu,17129.0
68,10175068,2012-04-16 13:27:44,15,7501,Select data at a particular level from a Multi...,2,3,5.0,elyase,19551.0,elyase,19551.0
74,10264739,2012-04-22 02:35:06,9,2362,major memory problems reading in a csv file us...,3,2,7.0,vgoklani,1752.0,vgoklani,1752.0
84,10475488,2012-05-07 00:38:47,5,2263,Calculating crossing (intercept) points of a S...,1,1,3.0,dailyglen,460.0,dailyglen,460.0
114,10771745,2012-05-27 04:53:45,0,1146,Removing duplicates from dataframe with index ...,1,2,1.0,dailyglen,460.0,dailyglen,460.0
169,11264307,2012-06-29 15:09:28,9,7436,"Adding levels to MultiIndex, removing without ...",1,0,4.0,Arthur G,987.0,Arthur G,987.0
195,11418192,2012-07-10 16:56:37,42,24109,pandas: complex filter on rows of DataFrame,4,0,6.0,duckworthd,4733.0,duckworthd,4733.0
239,11762815,2012-08-01 15:41:19,6,2945,How to resample a python pandas TimeSeries con...,2,1,NaN,THM,305.0,THM,305.0


### Exercise 4
<span  style="color:green; font-size:16px">Find all the questions that don't have an accepted answer, but have a score of more than 100</span>

In [14]:
so[so['ans_name'].isnull() & so['score'] > 100]

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep


### Exercise 5
<span  style="color:green; font-size:16px">Find all the questions where the reputation of the person asking the question is higher than the person answering it. Then find the percentage of times this happens</span>

In [24]:
rs = so[so['quest_rep'] > so['ans_rep']]
perc = (len(rs)/len(so))*100
print(rs,perc)

             id         creationdate  score  viewcount  \
26      9588331  2012-03-06 17:01:47     22      10038   
51      9962114  2012-04-01 05:25:20      9       8079   
99     10636024  2012-05-17 12:48:00     36      30386   
101    10671227  2012-05-20 06:14:57      0        473   
112    10760364  2012-05-25 19:33:39      8       2977   
123    10867028  2012-06-03 00:38:37     28      25115   
134    11004088  2012-06-12 20:11:53      5       2008   
137    11015974  2012-06-13 13:32:55      0        546   
182    11348183  2012-07-05 15:59:54     18      26272   
211    11586989  2012-07-20 21:11:28      8       9161   
218    11621165  2012-07-23 21:57:01      9      19512   
221    11640243  2012-07-24 22:30:16     34      18986   
255    11885916  2012-08-09 14:41:05      7        861   
259    11918342  2012-08-11 23:01:25      3       4829   
284    12047418  2012-08-21 01:41:25      3      19551   
311    12183432  2012-08-29 17:30:24      3       2285   
316    1219087

### Exercise 6
<span  style="color:green; font-size:16px">Find all the questions where the number of answers is between 5 and 10 inclusive, and the number of views is less than 1,000.</span>

In [26]:
so[(so['answercount'].between(5,10)) & (so['viewcount'] < 1000)]

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
620,13580875,2012-11-27 09:19:07,2,963,Running average / frequency of time series data?,5,9,1.0,Prof. Falken,15061.0,Victor K,332.0
1316,14742893,2013-02-07 03:13:31,2,598,Interpolation Function,5,0,NaN,eWizardII,886.0,NaN,NaN
4816,21315997,2014-01-23 18:02:57,2,716,Python Pandas vs R. Transformation Code Concis...,5,2,2.0,user2684301,1465.0,Jeff,62248.0
4958,19892107,2013-11-10 16:28:50,5,385,Updating csv with data from a csv with differe...,6,0,1.0,OliverSteph,32.0,NaN,NaN
5339,21040578,2014-01-10 09:33:09,-1,175,combining mutiple csv file,8,2,NaN,user104853,16.0,NaN,NaN
8842,27000092,2014-11-18 17:14:40,7,744,Count how many times each row is present in nu...,5,1,4.0,Akavall,30059.0,Jaime,42996.0
10525,29535163,2015-04-09 09:49:14,9,669,What dtype to use for money representation in ...,5,5,NaN,userqwerty1,362.0,NaN,NaN
10934,29407945,2015-04-02 08:04:51,8,997,Find and replace multiple values in python,9,1,1.0,blaz,945.0,Ashwini Chaudhary,154575.0
13789,30638568,2015-06-04 08:20:35,5,349,python if statement returns value error,5,0,NaN,hb.klein,151.0,EdChum,124367.0
14219,32724146,2015-09-22 18:22:24,2,395,Replacing zero values in dataframe using anoth...,5,2,NaN,user308827,2529.0,EdChum,124367.0


### Exercise 7
<span  style="color:green; font-size:16px">Find the inverse of exercise 6. Verify your results by adding the rows of both returned Series to see if it matches the number of rows of the original</span>

In [31]:
so[~(so['answercount'].between(5,10) & (so['viewcount'] < 1000))]

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
0,5486226,2011-03-30 12:26:50,4,2113,Rolling median in python,3,4,1.0,yueerhu,125.0,Mike Pennington,26995.0
1,5515021,2011-04-01 14:50:44,8,7015,Compute a compounded return series in Python,3,6,7.0,Jason Strimpel,3301.0,Mike Pennington,26995.0
2,5558607,2011-04-05 21:13:50,2,7392,Sort a pandas DataMatrix in ascending order,2,0,1.0,Jason Strimpel,3301.0,Wes McKinney,43310.0
3,6467832,2011-06-24 12:31:45,9,13056,How to get the correlation between two timeser...,1,0,7.0,user814005,117.0,Wes McKinney,43310.0
4,7577546,2011-09-28 01:58:38,9,2488,"Using pandas, how do I subsample a large DataF...",1,0,5.0,Uri Laserson,958.0,HYRY,54137.0
5,7766400,2011-10-14 10:33:54,2,977,Pandas + Django + mod_wsgi + virtualenv,1,2,1.0,Evan Davey,100.0,NaN,NaN
6,7776679,2011-10-15 08:21:17,25,28159,append two data frame with pandas,2,7,4.0,Jean-Pat,882.0,Wes McKinney,43310.0
7,7813132,2011-10-18 20:16:12,10,18917,Convert array of string (category) to array of...,3,0,6.0,Jean-Pat,882.0,Wes McKinney,43310.0
8,7837722,2011-10-20 14:46:14,201,223746,What is the most efficient way to loop through...,8,3,115.0,Muppet,1563.0,Nick Crawford,2779.0
9,8270129,2011-11-25 13:55:02,1,891,Pandas Panel for share portfolio,1,0,NaN,Evan Davey,100.0,Wes McKinney,43310.0


### Use the employee data for the rest of the exercises

In [32]:
employee = pd.read_csv('employee.csv')
employee.head()

,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,HIRE_DATE,JOB_DATE
0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,2006-06-12,2012-10-13
1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,2000-07-19,2010-09-18
2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,2015-02-03,2015-02-03
3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,1982-02-08,1991-05-25
4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,1989-06-19,1994-10-22


### Exercise 8
<span  style="color:green; font-size:16px">Find all the **`Black or African American`** females that work in the **`Houston Police Department-HPD`**</span>

In [36]:
employee[employee['RACE'] == 'Black or African American']

,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,HIRE_DATE,JOB_DATE
5,SENIOR POLICE OFFICER,Houston Police Department-HPD,66614.0,Black or African American,Full Time,Male,1984-11-26,2005-03-26
11,"CHIEF PHYSICIAN,MD",Health & Human Services,180416.0,Black or African American,Full Time,Male,1987-05-22,1999-08-28
12,CUSTOMER SERVICE REPRESENTATIVE I,Public Works & Engineering-PWE,30347.0,Black or African American,Full Time,Male,2015-11-16,2015-11-16
13,PUBLIC HEALTH INVESTIGATOR SUPERVISOR,Health & Human Services,55269.0,Black or African American,Full Time,Male,1999-10-20,2001-05-19
14,POLICE SERGEANT,Houston Police Department-HPD,77076.0,Black or African American,Full Time,Male,2001-06-04,2015-05-25
15,SENIOR POLICE OFFICER,Houston Police Department-HPD,NaN,Black or African American,Full Time,Male,1995-01-30,2015-02-14
21,FIRE FIGHTER,Houston Fire Department (HFD),57815.0,Black or African American,Full Time,Male,2001-03-20,2003-01-04
22,INSPECTOR,Public Works & Engineering-PWE,43264.0,Black or African American,Full Time,Male,2015-01-05,2015-01-05
24,POLICE OFFICER,Houston Police Department-HPD,52514.0,Black or African American,Full Time,Male,2010-03-29,2011-03-29
29,UTILITY WORKER,Public Works & Engineering-PWE,29557.0,Black or African American,Full Time,Male,2014-01-21,2014-01-21


### Exercise 9
<span  style="color:green; font-size:16px">Find the females that have a salary over 100,000 OR males with salary under 50,000</span>

In [40]:
employee[(employee['GENDER'] == 'Female') & (employee['BASE_SALARY'] > 100000)]

,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,HIRE_DATE,JOB_DATE
0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,2006-06-12,2012-10-13
66,"PUBLIC HEALTH DENTIST,DDS",Health & Human Services,100791.0,White,Full Time,Female,2015-12-28,2015-12-28
237,ASSISTANT DIRECTOR (EXECUTIVE LEVEL),Admn. & Regulatory Affairs,130416.0,Asian/Pacific Islander,Full Time,Female,2002-05-24,2013-07-20
366,DEPUTY ASSISTANT DIRECTOR (EXECUTIVE LEV,Mayor's Office,110000.0,White,Full Time,Female,2014-05-13,2014-05-13
522,DEPUTY ASSISTANT DIRECTOR (EX LVL),Houston Airport System (HAS),110686.0,Black or African American,Full Time,Female,2011-11-07,2011-11-07
591,SENIOR STAFF ANALYST,Public Works & Engineering-PWE,105503.0,White,Full Time,Female,1991-08-05,2015-05-23
605,DEPUTY DIRECTOR (EXECUTIVE LEVEL),Houston Airport System (HAS),150416.0,Black or African American,Full Time,Female,2013-06-10,2013-06-10
742,SENIOR ASSISTANT CITY ATTORNEY II,Legal Department,150416.0,White,Full Time,Female,2012-06-11,2013-07-06
793,DIVISION MANAGER,Houston Airport System (HAS),100873.0,White,Full Time,Female,2005-10-12,2005-10-12
815,SENIOR ASSISTANT CITY ATTORNEY II,Legal Department,126115.0,Hispanic/Latino,Full Time,Female,1996-10-30,2004-10-02


### Exercise 10
<span  style="color:green; font-size:16px">Find the females in the following departments with salary over 60,000 (Parks & Recreation, Solid Waste Management, Fleet Management Department, Library)  </span>

In [42]:
employee[(employee['BASE_SALARY'] > 60000) & (employee['DEPARTMENT'].isin(['Parks & Recreation', 'Solid Waste Management', 'Fleet Management Department', 'Library']))]

,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,HIRE_DATE,JOB_DATE
124,SOLID WASTE SUPERVISOR,Solid Waste Management,61428.0,Black or African American,Full Time,Male,1990-09-18,2004-12-11
249,STAFF ANALYST,Solid Waste Management,75041.0,Hispanic/Latino,Full Time,Female,1992-10-21,2015-07-18
412,ADMINISTRATIVE COORDINATOR,Library,79302.0,Hispanic/Latino,Full Time,Female,2003-12-22,2005-04-02
476,ADMINISTRATIVE SUPERVISOR,Library,60632.0,Black or African American,Full Time,Female,1993-05-10,2014-11-08
561,ASSISTANT DIRECTOR (EXECUTIVE LEVEL),Fleet Management Department,125884.0,White,Full Time,Male,2009-08-24,2016-02-13
892,LIBRARIAN III,Library,61454.0,White,Full Time,Female,1998-11-02,2002-06-15
1165,DEPUTY ASSISTANT DIRECTOR (EXECUTIVE LEV,Library,107763.0,Black or African American,Full Time,Female,1993-11-16,2014-03-15
1197,SUPERINTENDENT,Parks & Recreation,72280.0,Hispanic/Latino,Full Time,Male,1994-03-21,2011-08-06
1256,ADMINISTRATION MANAGER,Solid Waste Management,77966.0,White,Full Time,Male,1991-11-12,2011-11-12
1469,DIVISION MANAGER,Parks & Recreation,85055.0,White,Full Time,Male,1997-06-10,2013-04-13


### Exercise 11
<span  style="color:green; font-size:16px">Find all the males with salary over 100,000. Return only the race, gender and salary columns</span>

In [51]:
employee.loc[(employee['GENDER'] == 'Male') & (employee['BASE_SALARY'] > 100000),['RACE','GENDER','BASE_SALARY']]

,RACE,GENDER,BASE_SALARY
8,White,Male,107962.0
11,Black or African American,Male,180416.0
43,Hispanic/Latino,Male,165216.0
169,White,Male,120916.0
178,White,Male,210588.0
186,White,Male,110881.0
217,Asian/Pacific Islander,Male,102019.0
297,White,Male,141948.0
299,Black or African American,Male,100228.0
416,White,Male,104455.0


### Exercise 12
<span  style="color:green; font-size:16px">Select all salaries as a Series in a separate variable. From this series select all salaries under 25,000</span>

In [55]:
serie = employee['BASE_SALARY']
employee[serie >= 25000]

,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,HIRE_DATE,JOB_DATE
0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,2006-06-12,2012-10-13
1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,2000-07-19,2010-09-18
2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,2015-02-03,2015-02-03
3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,1982-02-08,1991-05-25
4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,1989-06-19,1994-10-22
5,SENIOR POLICE OFFICER,Houston Police Department-HPD,66614.0,Black or African American,Full Time,Male,1984-11-26,2005-03-26
6,ENGINEER,Public Works & Engineering-PWE,71680.0,Asian/Pacific Islander,Full Time,Male,2012-03-26,2012-03-26
7,CARPENTER,Houston Airport System (HAS),42390.0,White,Full Time,Male,2013-11-04,2013-11-04
8,DEPUTY ASSISTANT DIRECTOR (EXECUTIVE LEV,Public Works & Engineering-PWE,107962.0,White,Full Time,Male,1993-11-15,2013-01-05
9,AIRPORT OPERATIONS COORDINATOR,Houston Airport System (HAS),44616.0,NaN,Full Time,Male,2016-03-14,2016-03-14


### Exercise 13
<span  style="color:green; font-size:16px">Get the same exact result as exercise 11, but make your selection from the employee DataFrame. Use only a single line of code</span>

In [56]:
employee[employee['BASE_SALARY'] >= 25000]

,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,HIRE_DATE,JOB_DATE
0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,2006-06-12,2012-10-13
1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,2000-07-19,2010-09-18
2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,2015-02-03,2015-02-03
3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,1982-02-08,1991-05-25
4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,1989-06-19,1994-10-22
5,SENIOR POLICE OFFICER,Houston Police Department-HPD,66614.0,Black or African American,Full Time,Male,1984-11-26,2005-03-26
6,ENGINEER,Public Works & Engineering-PWE,71680.0,Asian/Pacific Islander,Full Time,Male,2012-03-26,2012-03-26
7,CARPENTER,Houston Airport System (HAS),42390.0,White,Full Time,Male,2013-11-04,2013-11-04
8,DEPUTY ASSISTANT DIRECTOR (EXECUTIVE LEV,Public Works & Engineering-PWE,107962.0,White,Full Time,Male,1993-11-15,2013-01-05
9,AIRPORT OPERATIONS COORDINATOR,Houston Airport System (HAS),44616.0,NaN,Full Time,Male,2016-03-14,2016-03-14
